In [11]:
import requests
import json
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.page import Page
import re

my_app_id = '915181757497456'
my_app_secret = 'ba3aa064b98d6aae80319d2acd3d27c1'
my_access_token = 'EAANAWluPgHABOyYb2dZC6It0siq9xIFLiUCZB8JPaBc1IOZAMEw3JaUtTOAmLHS0zBkkuLQxmmxj18A4hs50Qa82yyETIiOgPLwtsheTMd3ATwZBTJB6tlQHo0IEKRsN1CJEFNwZB2GSfUV7LvLrCBDyGtNDMFN2xUM3017BUgOBRULDCJVbVvwDhWhe9zhSzXF1WgZASbIZArCCmPsugqemtsGJZCL0KqwlcSQZD'

FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)

In [13]:
page_id = "CocaColaMx"
page = Page(page_id)

posts = page.get_posts(fields=["id", "message", "created_time"], params={"limit": 5})

for post in posts:
    print(post)

FacebookRequestError: 

  Message: Call was not successful
  Method:  GET
  Path:    https://graph.facebook.com/v21.0/CocaColaMx/posts
  Params:  {'limit': 5, 'fields': 'id,message,created_time', 'summary': 'true'}

  Status:  400
  Response:
    {
      "error": {
        "message": "(#10) This endpoint requires the 'pages_read_engagement' permission or the 'Page Public Content Access' feature. Refer to https://developers.facebook.com/docs/apps/review/login-permissions#manage-pages and https://developers.facebook.com/docs/apps/review/feature#reference-PAGES_ACCESS for details.",
        "type": "OAuthException",
        "code": 10,
        "fbtrace_id": "A6J-0sah6ry4YB5p3CoEVgZ"
      }
    }


In [4]:
facebook_links = [
    "https://www.facebook.com/CocaColaMx/?locale=es_LA",
    "https://www.facebook.com/CocaColaMx/",
    "https://www.facebook.com/cocacolafemsamexico/?locale=es_LA",
    "https://www.facebook.com/TheCocaColaCo/",
    "https://www.facebook.com/CocaColaUnitedStates/",
    "https://www.facebook.com/Coca-Cola/"
]

def extract_username(url):
    match = re.search(r"facebook\.com/([^/?]+)", url)
    return match.group(1) if match else None

usernames = list(set(filter(None, [extract_username(link) for link in facebook_links])))

def get_page_id(username, access_token):
    url = f"https://graph.facebook.com/v21.0/{username}?fields=id&access_token={access_token}"
    response = requests.get(url).json()
    return response.get("id")

In [5]:
def get_page_posts(page_id):
    try:
        page = Page(page_id)
        posts = page.get_posts(
            fields=[
                "id", "message", "created_time", 
                "reactions.summary(true)", "comments.summary(true)", 
                "shares", "permalink_url"
            ],
            params={"limit": 5}
        )
        post_data = []
        for post in posts:
            post_data.append({
                "id": post.get("id"),
                "message": post.get("message"),
                "created_time": post.get("created_time"),
                "likes": post.get("reactions", {}).get("summary", {}).get("total_count", 0),
                "comments": post.get("comments", {}).get("summary", {}).get("total_count", 0),
                "shares": post.get("shares", {}).get("count", 0),
                "link": post.get("permalink_url")
            })
        return post_data
    except Exception as e:
        return {"error": str(e)}

In [7]:
result_data = {}
for username in usernames:
    page_id = get_page_id(username, my_access_token)
    if page_id:
        result_data[username] = get_page_posts(page_id)
        print(page_id)
    else:
        result_data[username] = {"error": "No se pudo obtener el ID de la página"}


In [8]:
with open("facebook_posts.json", "w", encoding="utf-8") as file:
    json.dump(result_data, file, ensure_ascii=False, indent=4)

print("Datos guardados en facebook_posts.json")

Datos guardados en facebook_posts.json
